<a href="https://colab.research.google.com/github/KU-BIG/KUBIG_2023_SPRING/blob/main/1.%20%EB%B0%A9%ED%95%99%EB%B6%84%EB%B0%98/%EB%94%A5%EB%9F%AC%EB%8B%9D/%EA%B3%BC%EC%A0%9C/6%EC%A3%BC%EC%B0%A8/6%E1%84%8C%E1%85%AE%E1%84%8E%E1%85%A1_%EA%B9%80%EC%97%B0%EA%B7%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [ ]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [ ]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {val : i for i, val in enumerate(world_set)}

In [ ]:
print(vocab)

{'a': 0, 'c': 1, 'u': 2, 'b': 3, "'": 4, 'w': 5, 'B': 6, 'n': 7, 'g': 8, ' ': 9, 'd': 10, 't': 11, 'f': 12, 'm': 13, 'k': 14, 'i': 15, '.': 16, 'l': 17, 'y': 18, 'e': 19, 'h': 20, 'p': 21, ',': 22, 'r': 23, 's': 24, 'o': 25}


In [ ]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [ ]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [ ]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i+sequence_length]
  y_str = sentence[i+1 : i+1+sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [ ]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[6, 23, 15, 1, 14, 9, 5, 0, 17, 17]
[23, 15, 1, 14, 9, 5, 0, 17, 17, 24]


In [ ]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(len(vocab))[x] for x in x_data]
# np.eye(N, M=None) -> (N x M) 크기의 2차원 행렬을 만들어 주는 함수

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-9-9ec4e4519ecc>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  X = torch.FloatTensor(x_one_hot)


In [ ]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [ ]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [ ]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([23, 15,  1, 14,  9,  5,  0, 17, 17, 24])


In [ ]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, vocab_size, bias=True)
  
  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [ ]:
net = Net(vocab_size, hidden_size, 2)

In [ ]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [ ]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB
ouauunuouuaauouaooauauuounuouauouuauouaouaauuaoauooaouauonuuuoaouuaouauunuouuaauouuookouauauooauuuoauoauaoonuaoauuouauauuaukaaaooonoauoaouuaouauunuouuaauouaooauauooaaauonuaauuonuuoauaoouoouokouauoa
o a  aa  aaaao a     a o aao  ao  aa  a   au   a   a  a  a  a a a ao    aa  aaaao  o a     a  au   aa a    au  a  o   aa    aaao  a a  a a ao    aa  aaaao a     a  aaa  aaaaa  aa oa ao ao   a  a o 
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [ ]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thingsd"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [ ]:
sentence = ("A nice house sits on the western plains, surrounded by "
            "burrows, pyramid-shaped structures and long highways where lizards, "
            "squirrels, and hedgehogs can hide themselves.")

In [ ]:
world_set = list(set(sentence))

# 각 문자에 정수 인코딩
vocab = {val : i for i, val in enumerate(world_set)}

In [ ]:
print(vocab)

{'u': 0, '-': 1, 'l': 2, 'i': 3, 'p': 4, 'A': 5, 's': 6, 'r': 7, 'd': 8, 'v': 9, 'z': 10, ' ': 11, 'h': 12, 'n': 13, 'w': 14, 'o': 15, 'y': 16, 'c': 17, ',': 18, 'm': 19, 'e': 20, 't': 21, 'b': 22, '.': 23, 'q': 24, 'g': 25, 'a': 26}


In [ ]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 27


In [ ]:
hidden_size = vocab_size # 동일해야 함.
sequence_length = 10  # 너무 길거나 짧지 않게 해야 함.
learning_rate = 0.01

In [ ]:
# seqence 길이 단위 자르기

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1 : i+1+sequence_length]
    print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
    x_data.append([vocab[c] for c in x_str])
    y_data.append([vocab[d] for d in y_str])

0 A nice hou ->  nice hous
1  nice hous -> nice house
2 nice house -> ice house 
3 ice house  -> ce house s
4 ce house s -> e house si
5 e house si ->  house sit
6  house sit -> house sits
7 house sits -> ouse sits 
8 ouse sits  -> use sits o
9 use sits o -> se sits on
10 se sits on -> e sits on 
11 e sits on  ->  sits on t
12  sits on t -> sits on th
13 sits on th -> its on the
14 its on the -> ts on the 
15 ts on the  -> s on the w
16 s on the w ->  on the we
17  on the we -> on the wes
18 on the wes -> n the west
19 n the west ->  the weste
20  the weste -> the wester
21 the wester -> he western
22 he western -> e western 
23 e western  ->  western p
24  western p -> western pl
25 western pl -> estern pla
26 estern pla -> stern plai
27 stern plai -> tern plain
28 tern plain -> ern plains
29 ern plains -> rn plains,
30 rn plains, -> n plains, 
31 n plains,  ->  plains, s
32  plains, s -> plains, su
33 plains, su -> lains, sur
34 lains, sur -> ains, surr
35 ains, surr -> ins, surro
36

In [ ]:
# 입력 시퀀스에 대해 원핫인코딩 수행

# np.eye를 사용하여 x_data를 원핫인코딩
x_one_hot = [np.eye(len(vocab))[x] for x in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-8-1fb9ae18dd2a>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  X = torch.FloatTensor(x_one_hot)


In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([158, 10, 27])
레이블의 크기 : torch.Size([158, 10])


In [ ]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0

In [ ]:
print(Y[0])

tensor([11, 13,  3, 17, 20, 11, 12, 15,  0,  6])


### 1. RNN

In [ ]:
class Net1(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net1, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, vocab_size, bias=True)
  
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

### 2. LSTM

In [ ]:
class Net2(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net2, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers = layers,
                            batch_first = True, bidirectional = True)
        self.fc = torch.nn.Linear(hidden_size * 2, vocab_size, bias=True) # 양방향이므로
  
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

In [ ]:
net1 = Net1(vocab_size, hidden_size, 2)
net2 = Net2(vocab_size, hidden_size, 2)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net1.parameters(), learning_rate)

In [ ]:
outputs_1 = net1(X)
print(outputs_1.shape)

torch.Size([158, 10, 27])


In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net1(X)

    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

 nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide themselves.
 nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide themselves.
 nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and hong highways where lizards, squirrels, and he gehogs can hide themselves.
 nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide themselves.
 nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide themselves.
 nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide them

In [ ]:
predict_str

' nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide themselves.'

In [ ]:
optimizer = optim.Adam(net2.parameters(), learning_rate)

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net2(X)

    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

AAAuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu
ssaaaaaaaaaaaaaaaaaaaaaaaaaaaauauuaaaaaaaauuuuuaaaauuaaaauuuuauuuauauaaaaaaaaauaauuaauuuuuaaaaaaaaaaauaaaaaaaaauuuaaaauauuuaauauuuuauaaaaaaaauuaauaaaaaaaaaauaaauaaaaaa
ssaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
seeeeeeeaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeerreeeeeeeeeeeeeeeeeeeeeeeeeerrrreeeeeeereerrrreeeeeeereeeeeeeeeeeeeeerrrrreeeeeeeeeeeeeeeeeeeeeeeeeeeeeeereererrrrrreeeeeeeeeeeeeeeeeeereeeeeeeeeee

In [ ]:
predict_str

' nice house sits on the western plains, surrounded by burrows, pyramid-shaped structures and long highways where lizards, squirrels, and hedgehogs can hide themselves.'

- 직전 문자를 이용하여 다음 문자를 예측하는 방식에 따라 맨 첫 글자인 'A'를 제외하고 잘 예측되었다.
- 문장 길이가 짧아 RNN, LSTM의 예측 정도에 차이가 없었다.